In [1]:
import matplotlib.pyplot as plt
import pickle
import os
from itertools import combinations
import numpy as np
import collections
from IPython.lib import backgroundjobs as bg
from tqdm import tqdm_notebook
import heapq
from gensim.models import Word2Vec
import node2vec
import networkx as nx
from sklearn import metrics

In [2]:
with open('./data/2016_q1_cci-neighbours_kcore.pickle', 'rb') as picklefile:
    neighbours_q1 = pickle.load(picklefile)

In [3]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q1 = pickle.load(picklefile)

In [4]:
dimensions=32
window=10
iterations=10
p=1
q=1
num_walks=25
walk_length = 400

In [5]:
nx_weighted = nx.Graph()
nx_weighted.add_nodes_from(neighbours_q1.keys())
for edge in weights_q1:
    u, v = edge[0]
    nx_weighted.add_edge(u, v, weight=1)

In [6]:
G = node2vec.Graph(nx_weighted, False, p, q)
%time G.preprocess_transition_probs()

CPU times: user 17min 31s, sys: 7.82 s, total: 17min 39s
Wall time: 17min 40s


In [ ]:
%time walks = G.simulate_walks(num_walks, walk_length)
walks = [map(str, walk) for walk in walks]
%time model = Word2Vec(walks, size=dimensions, window=window, min_count=0, sg=1, iter=iterations)

Walk iteration:
1 / 25
2 / 25
3 / 25
4 / 25
5 / 25
6 / 25
7 / 25
8 / 25


In [ ]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q1 = pickle.load(picklefile)
with open('./data/2016_q2_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q2 = pickle.load(picklefile)
edges_q1 = set(edge for edge, weight in weights_q1)
edges_q2 = set(edge for edge, weight in weights_q2)
new_edges=edges_q2.difference(edges_q1)
k = len(new_edges)

In [ ]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

100, 25 (2864, 0.11310323039254404)
100, 100 (3459, 0.1282488598865448)

In [19]:
len(neighbours_q1.keys())

3053

In [ ]:
authors = neighbours_q1.keys()
pairs = [(u, v) if u < v else (v, u) for u, v in combinations(authors, 2)]
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

In [ ]:
len(project_authors_q1['author_id'].unique())

In [ ]:
pairs = list(combinations(authors, 2))
alldist = {}
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    alldist[(u, v)] = model.wv.distance(str(u), str(v))
with open('./data/node2vec-weighted-p10-q01-100-100.pickle', 'wb') as picklefile:
    pickle.dump(alldist, picklefile)

In [ ]:
authors = neighbours_q1.keys()
pairs = list(combinations(authors, 2))
truth = [edge in edges_q2 for edge in pairs]
scores = [model.wv.similarity(str(u), str(v)) for u, v in pairs]
fpr, tpr, thres = metrics.roc_curve(truth,scores)
metrics.auc(fpr,tpr), metrics.average_precision_score(truth,scores)

In [27]:
sum(truth)

99311

In [ ]:
authors = neighbours_q1.keys()
pairs = [(u, v) if u < v else (v, u) for u, v in combinations(authors, 2)]
truth = [edge in edges_q2 for edge in pairs]
minus_q2 = list(set(pairs).difference(edges_q2))
y_train = [edge in edges_q1 for edge in minus_q2]
x_train = [abs(model.wv.get_vector(str(u))- model.wv.get_vector(str(v))) for u, v in minus_q2]
edgerep = [abs(model.wv.get_vector(str(u))- model.wv.get_vector(str(v))) for u, v in pairs]
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(penalty='l2',C=0.001)
lg.fit(x_train,y_train)
lg_y_pred_est = lg.predict_proba(edgerep)[:,1]
fpr,tpr,thresholds = metrics.roc_curve(truth,lg_y_pred_est)
metrics.auc(fpr,tpr), metrics.average_precision_score(truth,lg_y_pred_est)